# Multi density clustering for evolving datastream
- [Core Mini Cluster](#Core-Mini-Cluster)
- [Offline Phase](#Offline-Phase)
- [Online Phase](#Online-Phase)

In [1]:
__author__='Black D Chase,MR-TLL'
__version__='0.0.1'

In [ ]:
import torch

# Grid

- Grid weight:
   $$
      W_g(t_c) = \sum_{x \subset g} 2^{-\lambda * (t_c - t_x )}
   $$
- Grid weight Update:
   $$
      W_g(t_p,t_c) = 2^{-\lambda * (t_c - t_x )}* w_g(t_p) + 1
   $$
- Maximum possible theoretrical Weight of all data points:
   $$
   w_{max} = \frac{1}{1-2^{-\lambda}}
   $$
- Time Quantum for Grid and CMC updates 
   $$
   t_{pt} = \frac{log_{2}{\frac{\alpha}{\alpha - N(1-2^{- \lambda})}}}{\lambda}
   $$
- Grid->CMC condition:
    $$
      n_g > 1 \text{ and } w_g \ge \frac{\alpha}{N(1-2^{-\lambda})}
    $$
    
- OWT parameter for GRID and CMC Updates with time  ::
    If W_g < OWT --> remove grid from grid list.
    $$
      OWT(t_c,t_p) = \frac{\alpha(1-2^{-\lambda(t_c - t_p +1)})}{N(1-2^{-\lambda*t_p})}
    $$

In [ ]:
class Grid:
    def __init__(self,location):
        self.n = 0
        self.t = 0
        self.w = 0
        self.mcd = None
        self.c = None
        self.status = False # Sporadic
        self.location = location
        # hast to be a touple
        
        self.storage = []
    
    def __hash__(self):
        return hash(self.location)
    
    def __eq__(self,other):
        return self.location==other.location
    
    def updateGS(self,t,w):
        self.n+=1
        self.t=t
        self.w=w
        pass
    
    def calMCD(self):
        pass
    
    def calCenter(self):
        pass
    
    def addPoint(self,point):
        self.storage.append(point)
    
    def getDistance(self,point):
        pass
    
    def flush(self):
        self.storage = []
        self.n = 0
        self.t = 0
        self.w = 0
        self.mcd = None
        self.c = None
        self.status = False # Sporadic
    pass

# Core Mini Cluster

- Parameters Of CMC when a grid is promoted to CMC:
  $$
  W_{cmc} = W_{g}
  $$
  $$
  C_{cmc} = \frac{\sum_{1}^{n} f(t_c - t_i)(p_i)}{w_{cmc}}
  $$
  $$
  r_{cmc} = \frac{\sum_{1}^{n} f(t_c - t_i)(distance(p_{ij},c_{cmc}))}{w_{cmc}}
  $$
  
- If $$ W_{cmc} <  \frac{\alpha}{N(1-2^{-\lambda})} $$
   Then remove cmc from cmc list

In [2]:
class CMC(Grid):
    pass

# Clusterer

In [3]:
class Cluster:
    def __init__(self):
        pass
    pass

# Online Phase

In [4]:
class Online(Cluster):
    def __init__(self,dim,alpha,lamda):
        super(Offline,self).__init__()
        self.datasetDimension=dim
        self.cmcList={} ## list which containg all the cmc clusters as objects.
        self.gridList={} ## list which containg all the grids as objects.
        self.alpha=alpha
        self.lamda=lamda
        self.N=N ## no. of density grids as given in the paper.
        self.ng=0 ## no of grids made till now
        pass
        
        
        
    def train(self):
        while(True):
            newPoint=self.readPointFromStream()
            if newPoint==None:
                break;
            cmcs=self.findNearestCmc(newPoint)
            if calculateDist(newPoint,cmcs)<=cmcs.mcd:
                cmcs.addPoint(newPoint)
            else:
                ## we now need to check in which grid the newPoint resides, then update the grid list accordingly
                
        
    pass

# Offline Phase

In [6]:
class Offline(Cluster):
    def __init__(self,α,𝜆,gridGaranuality,N):
        super(Offline,self).__init__()
        
        pass
    pass

In [ ]:
gridGranuality=25
dataset=read()
colsRange = []
for cols in dataset:
    colsRange.append((min(cols),max(cols)))
